In [1]:
require 'torch'
require 'image'
require 'xlua'
require 'csvigo'
dofile '/home/shyam/Downloads/Independent_study/exp_fer/torch_exp/util.lua'
require 'nn'
require 'optim';
Plot = require 'itorch.Plot';

In [25]:
------------------------------------- setting up the training data ---------------------------------------
local Img_filename = '/home/shyam/Downloads/Independent_study/exp_fer/trainImg/'
local x = util:load_data(Img_filename)
local m = csvigo.load({path = "/home/shyam/Downloads/Independent_study/exp_fer/torch_exp/training.csv", mode = "large"})
local l = 28700;
local labels = torch.Tensor(l)

local x_n = torch.Tensor(l,28,28)
print(#x_n)
for i = 1,l do
   x_n[i] = x[i]
end

for i = 1,l do
   labels[i] = m[i+1][1] + 1
end
local x_n = torch.reshape(x_n,l,1,28,28)
trainset = {
    size = l,
    data = x_n,
    label = labels
}


<interpreted> configuration:
 + __main__ = false
 + ext = jpg


<csv>	parsing file: /home/shyam/Downloads/Independent_study/exp_fer/torch_exp/training.csv	


<csv>	parsing done	


 28700
    28
    28
[torch.LongStorage of size 3]



In [26]:
------------------------------------- setting up the testing data ---------------------------------------
local Img_filename = '/home/shyam/Downloads/Independent_study/exp_fer/testImg/'
local x = util:load_data(Img_filename)
local m = csvigo.load({path = "/home/shyam/Downloads/Independent_study/exp_fer/torch_exp/test.csv", mode = "large"})
local l = 2000;
local labels = torch.Tensor(l)

local x_n = torch.Tensor(l,28,28)
print(#x)
for i = 1,l do
   x_n[i] = x[i]
end

for i = 1,l do
   labels[i] = m[i+1][1] + 1
end
local x_n = torch.reshape(x_n,2000,1,28,28)
testset = {
    size = l,
    data = x_n,
    label = labels
}

<interpreted> configuration:
 + __main__ = false
 + ext = jpg


<csv>	parsing file: /home/shyam/Downloads/Independent_study/exp_fer/torch_exp/test.csv	


<csv>	parsing done	
 3589
   28
   28
[torch.LongStorage of size 3]



In [27]:
local temp_mean = trainset.data[{{},{1},{},{}}]:mean()
local temp_std = trainset.data[{{},{1},{},{}}]:std()
trainset.data[{{},{1},{},{}}]:add(-temp_mean)
trainset.data[{{},{1},{},{}}]:div(temp_std)
    
local temp_mean = testset.data[{{},{1},{},{}}]:mean()
local temp_std = testset.data[{{},{1},{},{}}]:std()
testset.data[{{},{1},{},{}}]:add(-temp_mean)
testset.data[{{},{1},{},{}}]:div(temp_std)

In [28]:


net = nn.Sequential()                      -- Sequential container.

-- layer 1
net:add(nn.SpatialConvolution(1, 6, 5, 5)) -- 3 input depth (RGB), 6 outputs (6 filters), 5x5 convolution kernel
net:add(nn.ReLU())                         -- ReLU non-linearity 
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))  -- A max-pooling operation that looks at 2x2 windows and finds the max.

-- layer 2
net:add(nn.SpatialConvolution(6, 16, 5, 5)) -- 6 input depth, 16 outputs (16 filters), 5x5 convolution kernel
net:add(nn.ReLU())                          
net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

-- layer 3
net:add(nn.View(16*4*4))                   -- **reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5.                                                 
net:add(nn.Linear(16*4*4, 120))            -- fully connected layer (16*5*5 inputs, 120 outputs)
net:add(nn.ReLU())                         -- non-linearity 

-- layer 4
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())                         -- non-linearity 

-- layer 5
net:add(nn.Linear(84, 7))                 -- 10 is the number of outputs of the network (in this case, 10 classes)

-- converts output to probabilities
net:add(nn.LogSoftMax())                    -- converts the output to a log-probability. Useful for classification problems

criterion = nn.ClassNLLCriterion()

-- print the network
print(net:__tostring());


nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> output]
  (1): nn.SpatialConvolution(1 -> 6, 5x5)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(2x2, 2,2)
  (4): nn.SpatialConvolution(6 -> 16, 5x5)
  (5): nn.ReLU
  (6): nn.SpatialMaxPooling(2x2, 2,2)
  (7): nn.View(256)
  (8): nn.Linear(256 -> 120)
  (9): nn.ReLU
  (10): nn.Linear(120 -> 84)
  (11): nn.ReLU
  (12): nn.Linear(84 -> 7)
  (13): nn.LogSoftMax
}	


In [29]:
input = torch.rand(1,1,28, 28)
scores = net:forward(input)
dummy_output_label = 3 
loss = criterion:forward(scores, dummy_output_label)
print('Scores:')
print(scores)
print('Loss:', loss)

Scores:	
-1.8633
-1.9388
-1.8281
-1.9774
-2.0293
-2.0458
-1.9582
[torch.DoubleTensor of size 7]

Loss:	1.8280591361405	


# print(net:__tostring());

In [30]:
sgd_params = {                              -- parameters for SGD
   learningRate = 1e-3,
   learningRateDecay = 1e-4
   --weightDecay = 1e-3,
   --momentum = 1e-4
}
x, dl_dx = net:getParameters()

In [31]:
step = function(batch_size)
    
    local time = sys.clock()
    
    local current_loss = 0
    local count = 0
    local shuffle = torch.randperm(trainset.size)
    batch_size = batch_size or 200
    
     -- 1. setup inputs and targets for this mini-batch
    
    for t = 1,trainset.size,batch_size do       
        
        local size = math.min(t + batch_size - 1, trainset.size) - t
        local inputs = torch.Tensor(size,1, 28, 28)
        local targets = torch.Tensor(size)
        
        for i = 1,size do
            local input = trainset.data[shuffle[i+t]]
            local target = trainset.label[shuffle[i+t]]

            inputs[i] = input
            targets[i] = target
        end
        --targets:add(1)
        
         --inputs = inputs:cuda()
         --targets = targets:cuda()        
        
        
        local feval = function(x_new)
            collectgarbage()
            
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero() -- reset gradients to zero

            
            -- perform mini-batch gradient descent
            
            -- 2. evaluate the loss function and its derivative with respect to x, given a mini batch
            -- backpropagation
            
            local outputs = net:forward(inputs)
            
            local loss = criterion:forward(outputs, targets)
            
            local df_do = criterion:backward(outputs, targets)
           
            net:backward(inputs, df_do)
            
                        
            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        
        count = count + 1
        current_loss = current_loss + fs[1]
    end

    time = sys.clock() - time
    time = time / trainset.size
    print("<trainer> time to learn 1 sample = " .. (time*1000) .. ' ms')
    
    -- normalize loss
    return current_loss / count
end

In [32]:
eval = function(dataset, batch_size)
    local count = 0
    batch_size = batch_size or 20
    
    for i = 1,dataset.size,batch_size do
        local size = math.min(i + batch_size - 1, dataset.size) - i
        local inputs = dataset.data[{{i,i+size-1}}]
        local targets = dataset.label[{{i,i+size-1}}]:long()
       
         --inputs = inputs:cuda()
         --targets = targets:cuda() 
        
        local predicts = net:forward(inputs)
        local _, indices = torch.max(predicts, 2)           -- _ is corresponding values
        --print(indices[{{1,10}}])
                                                            -- range of values in indices in 1 t0 10
        --indices:add(-1)                                     -- range of values in targets is 0 to 9
                                                                      
        local guessed_right = indices:eq(targets):sum()                 
        
        count = count + guessed_right
        
    end 
       
    return 100 * count / dataset.size
end

In [33]:
max_iter = 10     -- max iterations to run
x_axis = torch.FloatTensor(max_iter):fill(0.0)             
y_axis = torch.FloatTensor(max_iter):fill(0.0)             -- holds training loss
y_axis_val = torch.FloatTensor(max_iter):fill(0.0)         -- validation accuracy

do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    
    for i = 1,max_iter do
        local loss = step()
        print(string.format('Iteration: %d Current loss: %4f', i, loss)) 
        --itorch.image(confusion:render())
        local accuracy = eval(testset)
        print(string.format('Accuracy on the validation set: %4f', accuracy))

        
        x_axis[i] = i
        y_axis[i] = loss
        y_axis_val[i] = accuracy
        
        if accuracy < last_accuracy then
            if decreasing > threshold then break end
            decreasing = decreasing + 1
        else
            decreasing = 0
        end
        last_accuracy = accuracy
        
        -- display plot
        if i==1 then
            p1 = Plot():circle(x_axis[{{1, i}}], y_axis[{{1, i}}], 'red', 'hi'):draw()
            p1:title('Training Progression'):redraw()
            p1:xaxis('Iteration -->'):yaxis('Training loss -->'):redraw()
            p2 = Plot():circle(x_axis[{{1, i}}], y_axis_val[{{1, i}}], 'red', 'hi'):draw()
            p2:title('Validation Progression'):redraw()
            p2:xaxis('Iteration -->'):yaxis('Validation accuracy -->'):redraw()
        else
            p1:circle(x_axis[{{1, i}}], y_axis[{{1, i}}], 'red', 'hi'):redraw()
            p2:circle(x_axis[{{1, i}}], y_axis_val[{{1, i}}], 'red', 'hi'):redraw()
        end
        
    end
end


<trainer> time to learn 1 sample = 0.81687020923202 ms	
Iteration: 1 Current loss: 1.948386	


Accuracy on the validation set: 14.000000	


<trainer> time to learn 1 sample = 0.79592665728792 ms	
Iteration: 2 Current loss: 1.940225	


Accuracy on the validation set: 13.950000	


<trainer> time to learn 1 sample = 0.78711473983339 ms	
Iteration: 3 Current loss: 1.932362	


...

Accuracy on the validation set: 22.500000	


<trainer> time to learn 1 sample = 1.0771239676127 ms	
Iteration: 9 Current loss: 1.885921	


Accuracy on the validation set: 22.500000	


<trainer> time to learn 1 sample = 0.91723835842119 ms	
Iteration: 10 Current loss: 1.877914	


Accuracy on the validation set: 22.500000	
